In [2]:
import string
import spacy
from tqdm import tqdm
import numpy as np
import pandas as pd
import json
from collections import defaultdict, Counter
import sys
from IPython.display import display
from random import randint
from copy import deepcopy
tqdm.pandas()
nlp = spacy.load('en_core_web_sm')
from utils import *

def addPunct(doc):
    if any(x in string.punctuation for x in doc[-3:]):
        return doc
    return doc +'.'
                

In [2]:
index = randint(0, 240)
table = pd.read_json('Data/DEV CQR Table.json').iloc[index:index+1]
turns = table.Reformulated_turns
slots = table.Slots
slots.iloc[0][list(slots.iloc[0].keys())[0]].append('Random property')
display(index, *slots, *turns, table)
results = [rolebased(x[:], y, True) for x, y in tqdm(zip(turns, slots))]
display(*results, *slots)
print(sum([sum([not(entity) for entity in entities ]) for entities in slots.iloc[0].values()]))

53

{'date': ['this week', 'Random property'],
 'weather_attribute': ['rain'],
 'location': ['menlo park']}

['is it going to rain sometime this week.',
 "What city are you wanting to know if it's going to rain?",
 'is it going to rain sometime this week in Menlo Park.',
 'In Menlo Park, it will rain on Tuesday and Wednesday.',
 'Thanks!',
 'No problem.']

,Original_turns,Original_RoleBased,Reformulated_turns,Reformulated_RoleBased,Slots
146,"[is it going to rain sometime this week., What...",[is it going to weather_attribute_1 sometime d...,"[is it going to rain sometime this week., What...",[is it going to weather_attribute_1 sometime d...,"{'date': ['this week', 'Random property'], 'we..."


1it [00:00,  6.87it/s]


['is it going to weather_attribute_1 sometime date_1.',
 "what city are you wanting to know if it's going to weather_attribute_1?",
 'is it going to weather_attribute_1 sometime date_1 in location_1.',
 'in location_1, it will weather_attribute_1 on tuesday and wednesday.',
 'thanks!',
 'no problem.']

{'date': ['this week', (False, 'Random property')],
 'weather_attribute': ['rain'],
 'location': ['menlo park']}

0


In [8]:
files = [('train', 'cqr_kvret_train_public.json'),
         ('test', 'cqr_kvret_test_public.json'),
         ('dev', 'cqr_kvret_dev_public.json')]
data = defaultdict(list)

for dataset, filename in files:
    print("\n{}\n______\n".format(dataset.upper()))
    with open(filename, 'r') as file:
        data[dataset] = json.load(file)
    slotsData = []
    #puntuating the turns and combining dialogues into one for sequence to sequence model input
    inputDataArray = [[addPunct(turn['data']['utterance']) for turn in dialogue['dialogue']] for dialogue in data[dataset]]
    domain = []
    for dialogue in data[dataset]:
        slot_set = set()
        temp_slots = defaultdict(list)
        domain.append(dialogue['scenario']['task']['intent'])
#         continue
        for turn in dialogue['dialogue']:
            slots = list(turn['data'].setdefault('slots', {}).items())
            slots.extend(list(turn.setdefault('reformulation', {}).setdefault('slots', {}).items()))
            for slot in slots:
                for slot in slotgenerator(slot):
                    if slot in slot_set:
                        continue
                    slot_set.add(slot)
                    temp_slots[slot[0]].append(slot[1])
        slotsData.append(temp_slots)
#     display(domain)
#     continue
#     sys.exit()
    outputDataArray = []
    for dialogue in data[dataset]:
        text = [addPunct(turn['data']['utterance']) for turn in dialogue['dialogue']]
        for i, x in enumerate(text):
            if dialogue['dialogue'][i]['reformulation'].setdefault('reformulated_utt', None):
                text[i-1] = dialogue['dialogue'][i]['reformulation']['reformulated_utt']
                text[i-1] = addPunct(text[i-1])
        outputDataArray.append(text)
    DataTable = {'Original_turns': inputDataArray,
                 'Original_EntityIndex':[rolebased(x[:], y, slot_type=False) for x, y in tqdm(zip(inputDataArray, slotsData))],
                 'Original_RoleBased':[rolebased(x[:], y) for x, y in tqdm(zip(inputDataArray, slotsData))],
                 'Original_RoleBased_INVERSE':[[x[:], y] for x, y in tqdm(zip(inputDataArray, slotsData))],
                 'Reformulated_turns': outputDataArray, 
                 'Reformulated_EntityIndex':[rolebased(x[:], y, slot_type=False) for x, y in tqdm(zip(outputDataArray, slotsData))], 
                 'Reformulated_RoleBased_INVERSE':[[x[:], y] for x, y in tqdm(zip(outputDataArray, slotsData))],
                 'Reformulated_RoleBased':[rolebased(x[:], y) for x, y in tqdm(zip(outputDataArray, slotsData))],
                 'Slots': slotsData,
                 'Domain': domain
                }
#     sys.exit()
    unused_slots = sum([sum([sum([type(entity)!=str for entity in entities ]) for entities in slots.values()]) for slots in slotsData])
    all_slots = sum([sum([len(entities) for entities in slots.values()]) for slots in slotsData])
    print(unused_slots, all_slots, unused_slots/all_slots)
    table = pd.DataFrame.from_dict(DataTable)
    display(table.head())
    table.to_json('Data/Table/{} CQR Table.json'.format(dataset.upper()))
        


TRAIN
______



2131it [08:11,  5.14it/s]
2131it [08:28,  4.03it/s]
2131it [00:00, 267342.50it/s]
2131it [08:02,  6.07it/s]
2131it [00:00, 213665.66it/s]
2131it [06:29,  5.80it/s]


0 9861 0.0


,Original_turns,Original_EntityIndex,Original_RoleBased,Original_RoleBased_INVERSE,Reformulated_turns,Reformulated_EntityIndex,Reformulated_RoleBased_INVERSE,Reformulated_RoleBased,Slots,Domain
0,"[where's the nearest parking garage., The near...","[where's the entity_2 entity_4., the entity_2 ...","[where's the distance_1 poi_type_1., the dista...","[[where's the nearest parking garage., The nea...","[where's the nearest parking garage., The near...","[where's the entity_2 entity_4., the entity_2 ...","[[where's the nearest parking garage., The nea...","[where's the distance_1 poi_type_1., the dista...","{'distance': ['nearest'], 'poi_type': ['parkin...",navigate
1,"[Where is the nearest gas station?, There is a...","[where is the entity_1 entity_4?, there is a e...","[where is the distance_1 poi_type_1?, there is...","[[Where is the nearest gas station?, There is ...","[Where is the nearest gas station?, There is a...","[where is the entity_1 entity_4?, there is a e...","[[Where is the nearest gas station?, There is ...","[where is the distance_1 poi_type_1?, there is...","{'distance': ['nearest', '4 miles away'], 'poi...",navigate
2,[show me the closest location where i can get ...,[show me the entity_1 location where i can get...,[show me the distance_1 location where i can g...,[[show me the closest location where i can get...,[show me the closest location where i can get ...,[show me the entity_1 location where i can get...,[[show me the closest location where i can get...,[show me the distance_1 location where i can g...,"{'distance': ['closest', '5 miles away'], 'poi...",navigate
3,"[What time is dinner and who is it with?, Dinn...","[what time is entity_1 and who is it with?, en...","[what time is event_1 and who is it with?, eve...","[[What time is dinner and who is it with?, Din...","[What time is dinner and who is it with?, Dinn...","[what time is entity_1 and who is it with?, en...","[[What time is dinner and who is it with?, Din...","[what time is event_1 and who is it with?, eve...",{'event': ['dinner']},schedule
4,"[schedule dentist appointment., when is your d...","[schedule entity_2., when is your entity_2., i...","[schedule event_1., when is your event_1., it ...","[[schedule dentist appointment., when is your ...","[schedule dentist appointment., when is your d...","[schedule entity_2., when is your entity_2., s...","[[schedule dentist appointment., when is your ...","[schedule event_1., when is your event_1., sch...","{'event': ['dentist appointment'], 'date': ['s...",schedule



TEST
______



...

,Original_turns,Original_EntityIndex,Original_RoleBased,Original_RoleBased_INVERSE,Reformulated_turns,Reformulated_EntityIndex,Reformulated_RoleBased_INVERSE,Reformulated_RoleBased,Slots,Domain
0,"[remind me to take my pills., What time do you...","[remind me to entity_1., what time do you need...","[remind me to event_1., what time do you need ...","[[remind me to take my pills., What time do yo...","[remind me to take my pills., What time do you...","[remind me to entity_1., what time do you need...","[[remind me to take my pills., What time do yo...","[remind me to event_1., what time do you need ...","{'event': ['take pills'], 'time': ['7pm']}",schedule
1,"[What gas stations are here?, There is a Chevr...","[what entity_3 are here?, there is a entity_2....","[what poi_type_1 are here?, there is a poi_1.,...","[[What gas stations are here?, There is a Chev...","[What gas stations are here?, There is a Chevr...","[what entity_3 are here?, there is a entity_2....","[[What gas stations are here?, There is a Chev...","[what poi_type_1 are here?, there is a poi_1.,...","{'poi_type': ['gas stations', 'gas station'], ...",navigate
2,[what's the temperature going to be like on fr...,[what's the entity_3 going to be like on entit...,[what's the weather_attribute_1 going to be li...,[[what's the temperature going to be like on f...,[what's the temperature going to be like on fr...,[what's the entity_3 going to be like on entit...,[[what's the temperature going to be like on f...,[what's the weather_attribute_1 going to be li...,"{'date': ['friday'], 'weather_attribute': ['te...",weather
3,[give me directions to the closest grocery sto...,"[give me directions to the closest entity_5., ...",[give me directions to the closest poi_type_1....,[[give me directions to the closest grocery st...,[give me directions to the closest grocery sto...,"[give me directions to the closest entity_5., ...",[[give me directions to the closest grocery st...,[give me directions to the closest poi_type_1....,"{'poi_type': ['grocery store'], 'traffic_info'...",navigate
4,[What will the weather in Fresno be in the nex...,[what will the weather in entity_2 be in the e...,[what will the weather in location_1 be in the...,[[What will the weather in Fresno be in the ne...,[What will the weather in Fresno be in the nex...,[what will the weather in entity_2 be in the e...,[[What will the weather in Fresno be in the ne...,[what will the weather in location_1 be in the...,"{'date': ['next 48 hours'], 'location': ['fres...",weather



DEV
______



271it [00:40,  6.71it/s]
271it [00:49,  5.42it/s]
271it [00:00, 90505.33it/s]
271it [00:57,  4.75it/s]
271it [00:00, 270890.46it/s]
271it [00:54,  4.97it/s]


0 1045 0.0


,Original_turns,Original_EntityIndex,Original_RoleBased,Original_RoleBased_INVERSE,Reformulated_turns,Reformulated_EntityIndex,Reformulated_RoleBased_INVERSE,Reformulated_RoleBased,Slots,Domain
0,[make an appointment to reserve conference roo...,[make an appointment to reserve entity_4 later...,[make an appointment to reserve room_1 later t...,[[make an appointment to reserve conference ro...,[make an appointment to reserve conference roo...,[make an appointment to reserve entity_4 later...,[[make an appointment to reserve conference ro...,[make an appointment to reserve room_1 later t...,"{'agenda': ['for a meeting'], 'event': ['meeti...",schedule
1,[Is there supposed to be a blizzard in brentwo...,[is there supposed to be a entity_3 in entity_...,[is there supposed to be a weather_attribute_1...,[[Is there supposed to be a blizzard in brentw...,[Is there supposed to be a blizzard in brentwo...,[is there supposed to be a entity_3 in entity_...,[[Is there supposed to be a blizzard in brentw...,[is there supposed to be a weather_attribute_1...,"{'date': ['friday'], 'location': ['brentwood']...",weather
2,"[Address to the gas station ., Valero is locat...","[address to the entity_3., entity_2 is located...","[address to the poi_type_1., poi_1 is located ...","[[Address to the gas station ., Valero is loca...","[Address to the gas station ., Valero is locat...","[address to the entity_3., entity_2 is located...","[[Address to the gas station ., Valero is loca...","[address to the poi_type_1., poi_1 is located ...","{'poi_type': ['gas station'], 'traffic_info': ...",navigate
3,"[get me directions to the nearest starbucks., ...","[get me directions to the entity_6 entity_3., ...",[get me directions to the traffic_info_1 poi_1...,"[[get me directions to the nearest starbucks.,...","[get me directions to the nearest starbucks., ...","[get me directions to the entity_6 entity_3., ...","[[get me directions to the nearest starbucks.,...",[get me directions to the traffic_info_1 poi_1...,"{'poi': ['starbucks', 'coupa'], 'traffic_info'...",navigate
4,"[remind me of the time for today's dinner., Di...","[remind me of the time for today's entity_2., ...","[remind me of the time for today's event_1., e...","[[remind me of the time for today's dinner., D...","[remind me of the time for today's dinner., Di...","[remind me of the time for today's entity_2., ...","[[remind me of the time for today's dinner., D...","[remind me of the time for today's event_1., e...","{'date': ['wednesday'], 'event': ['dinner'], '...",schedule


In [4]:
table = pd.read_json('Data\Table\TREC+ CQR Table.json')
slotsData = []
for dialogslots in table.Slots:
    temp = defaultdict(list)
    slotset = set()
    for slot_slottype in dialogslots:
        for all_slots in [slotgenerator(x) for x in list(slot_slottype.items())]:
            for slot in all_slots:
                if slot[1] not in slotset:
                    slotset.add(slot[1])
                    temp[slot[0]].append(slot[1])
#                 print(slot)
    slotsData.append(temp)
table['Original_EntityIndex'] = [rolebased(x[:], y, slot_type=False, ordered=True) for x, y in tqdm(zip(table.Original_no_abs, slotsData))]
table['Reformulated_EntityIndex'] = [rolebased(x[:], y, slot_type=False, ordered=True) for x, y in tqdm(zip(table.Reformulated_turns, slotsData))]
display(*table.iloc[8], slotsData[8])
table.to_json('Data\Table\TREC++ CQR Table.json')

30it [00:02,  9.50it/s]
30it [00:03,  8.63it/s]


['What are the different forms of energy?',
 'How can it be stored?',
 'What type of energy is used in motion?',
 'Tell me about mechanical energy.',
 'Give me some examples.',
 'Why is sound a form of mechanical energy?',
 'How does it differ from potential energy?',
 'Are potential and kinetic the same?',
 'What type does chemical energy belong to?',
 'What form of energy is used in eating?']

['What are the different forms of energy?',
 'How can energy be stored?',
 'What type of energy is used in motion?',
 'Tell me about mechanical energy.',
 'Give me some examples of mechanical energy',
 'Why is sound a form of mechanical energy?',
 'How does mechanical energy differ from potential energy?',
 'Are potential and kinetic energy the same?',
 'What type of energy does chemical energy belong to?',
 'What form of energy is used in eating?']

[{},
 {'it': ['energy']},
 {},
 {},
 {'': ['mechanical energy']},
 {},
 {'it': ['mechanical energy']},
 {},
 {'': ['energy']},
 {}]

['What are the different forms of energy?',
 'How can it be stored?',
 'What type of energy is used in motion?',
 'Tell me about mechanical energy.',
 'Give me some examples.',
 'Why is sound a form of mechanical energy?',
 'How does it differ from potential energy?',
 'Are potential and kinetic the same?',
 'What type does chemical energy belong to?',
 'What form of energy is used in eating?']

['what are the different forms of entity_2?',
 'how can it be stored?',
 'what type of entity_2 is used in motion?',
 'tell me about entity_1.',
 'give me some examples.',
 'why is sound a form of entity_1?',
 'how does it differ from potential entity_2?',
 'are potential and kinetic the same?',
 'what type does chemical entity_2 belong to?',
 'what form of entity_2 is used in eating?']

['what are the different forms of entity_2?',
 'how can entity_2 be stored?',
 'what type of entity_2 is used in motion?',
 'tell me about entity_1.',
 'give me some examples of entity_1',
 'why is sound a form of entity_1?',
 'how does entity_1 differ from potential entity_2?',
 'are potential and kinetic entity_2 the same?',
 'what type of entity_2 does chemical entity_2 belong to?',
 'what form of entity_2 is used in eating?']

defaultdict(list, {'it': ['energy'], '': ['mechanical energy']})

In [12]:
# to test evaluation set
def evalSet(Islots, Oslots, table, maxD=3):
    for i, o in zip(Islots, Oslots):
        for d in range(maxD+1):
            eSet = i[d].intersection(o)
            table[d].append(eSet)
            
index = randint(0,240) #[randint(0,240) for _ in range(2)]
print(index)
for dataset in ['DEV', 'Data/DEV']:
    if dataset == 'DEV':
        ptable = pd.read_json(dataset +' CQR Table.json').iloc[index:index+1]
    table = pd.read_json(dataset +' CQR Table.json').iloc[index:index+1]
    inputDslots = table.Original_RoleBased.apply(dialog2TurnInputEval)
    outputDsolts = table.Reformulated_RoleBased.apply(dialog2TurnOutputEval)

    evaluation = defaultdict(list)
    for Islots, Oslots in zip(inputDslots, outputDsolts):
        evalSet(Islots, Oslots, evaluation)
    eTable = pd.DataFrame.from_dict(evaluation)
    display(dataset, eTable)
display('DEV', *ptable.Reformulated_RoleBased ,"Data/DEV", *table.Reformulated_RoleBased)
display('DEV', *ptable.Original_RoleBased ,"Data/DEV", *table.Original_RoleBased)
display('Original', *ptable.Original_turns ,"Reformulated", *table.Reformulated_turns)
display('Slots', *table.Slots)

64


'DEV'

,0,1,2,3
0,"{distance_1, poi_type_1}",{},{},{}
1,{distance_1},"{address_1, distance_1, poi_1}","{distance_1, poi_type_1}",{}


'Data/DEV'

,0,1,2,3
0,"{distance_1, poi_type_1}",{},{},{}
1,{distance_2},"{distance_3, poi_1, address_1}",{poi_type_1},{}


...

['find me a nearby coffee shop where i can sit and have some coffee.',
 'The Palo Alto Cafe is located at 436 Alger Dr, it is 2 miles away with moderate traffic.',
 "That's great! Please send me its address on screen and pick the quickest way for me.",
 "I sent the info on your screen, you're welcome!"]

'Reformulated'

['find me a nearby coffee shop where i can sit and have some coffee.',
 'The Palo Alto Cafe is located at 436 Alger Dr, it is 2 miles away with moderate traffic.',
 "That's great! Please send me the address of the coffee shop Palo Alto Cafe 2 miles away at 436 Alger Dr on screen and pick the quickest way to the coffee shop Palo Alto Cafe 2 miles away at 436 Alger Dr for me.",
 "I sent the info on your screen, you're welcome!"]

'Slots'

{'distance': ['nearby', 'quickest way', '2 miles away'],
 'poi_type': ['coffee shop'],
 'address': ['436 alger dr'],
 'poi': ['palo alto cafe']}

## Dialog to Turn

In [2]:
# Creating evaluation files
def evalSet(Islots, Oslots, index, domain, table, maxD=3):
    for i, o in zip(Islots, Oslots):
        table['SlotIndex'].append(index)
        for d in range(maxD+1):
            eSet = i[d].intersection(o)
            table[d].append(eSet)
        table['Domain'].append(domain)
            
for dataset in ['TRAIN', 'TEST', 'DEV']:
    table = pd.read_json('Data/Table/' + dataset +' CQR Table.json')    
    
    inputDslots = table.Original_RoleBased.progress_apply(dialog2TurnInputEval)
    outputDsolts = table.Reformulated_RoleBased.progress_apply(dialog2TurnOutputEval)
    
    evaluation = defaultdict(list)
    for i, (Islots, Oslots, domain) in tqdm(enumerate(zip(inputDslots, outputDsolts, table.Domain))):
        evalSet(Islots, Oslots, i, domain, evaluation)
    eTable = pd.DataFrame.from_dict(evaluation)
    display(dataset, eTable.head())
    eTable.to_json("Data/Evaluation/{} EVAL Table.json".format(dataset))

100%|████████████████████████████████████████████████████████| 2131/2131 [02:09<00:00, 16.04it/s]
2131it [00:00, 39545.80it/s]


'TRAIN'

,SlotIndex,0,1,2,3,Domain
0,0,"{distance_1, poi_type_1}",{},{},{},navigate
1,0,{},"{distance_1, poi_type_1, poi_1, address_1}","{distance_1, poi_type_1}",{},navigate
2,0,{},{},{},{},navigate
3,1,"{distance_1, poi_type_1}",{},{},{},navigate
4,1,{},"{distance_2, poi_1}",{poi_type_1},{},navigate


100%|██████████████████████████████████████████████████████████| 276/276 [00:16<00:00, 17.71it/s]
276it [00:00, 25157.07it/s]


'TEST'

,SlotIndex,0,1,2,3,Domain
0,0,{event_1},{},{},{},schedule
1,0,"{event_1, time_1}",{event_1},{event_1},{},schedule
2,1,{poi_type_1},{},{},{},navigate
3,1,"{distance_1, traffic_info_1}",{poi_1},{},{},navigate
4,1,{},{poi_1},{},{poi_1},navigate


100%|██████████████████████████████████████████████████████████| 271/271 [00:16<00:00, 15.34it/s]
271it [00:00, 24727.66it/s]


'DEV'

,SlotIndex,0,1,2,3,Domain
0,0,"{agenda_1, room_1}",{},{},{},schedule
1,0,"{date_1, time_1}",{},"{agenda_1, room_1}",{},schedule
2,1,"{location_1, weather_attribute_1, date_1}",{},{},{},weather
3,1,{},{},{},{},weather
4,2,"{distance_1, poi_type_1}",{},{},{},navigate


In [5]:
fileextension = 'Entity_index'
def evalSet(Islots, Oslots, table, maxD=3):
    for i, o in zip(Islots, Oslots):
        for d in range(maxD+1):
            eSet = i[d].intersection(o)
            table[d].append(eSet)

for dataset in ['TRAIN', 'TEST', 'DEV', 'TREC++'][3:]:
    table = pd.read_json('Data/Table/'+dataset +' CQR Table.json')
    with open('Data/Model/'+dataset + '_'+fileextension+'_input.txt', 'w') as file:
        file.write("\n".join(table
                             .Original_EntityIndex
                             .progress_apply(dialogs2TurnInput)
                             .apply(lambda x: "\n".join(x))
                            )
                  )
    with open('Data/Model/Output/'+dataset + '_'+fileextension+'_output.txt', 'w') as file:
        file.write("\n".join(table
                             .Reformulated_EntityIndex
                             .progress_apply(dialogs2TurnOutput)
                             .apply(lambda x: "\n".join(x))
                            )
                  )

100%|█████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 29599.89it/s]


In [7]:
table = pd.read_json('Data/Table/DEV CQR Table.json')
test = dialogs2TurnInput(table.iloc[2][3],index=False)
display(test, dialogs2TurnInput(table.iloc[2][2]), dialogs2TurnOutput(table.iloc[2][6], index=False))

['<user> give me directions to distance_1 poi_type_1.',
 '<user> give me directions to distance_2 poi_type_1. <system> there is poi_1 distance_1 from here but with heavy traffic on our way. <user> give me the address.',
 '<user> give me directions to distance_2 poi_type_1. <system> there is poi_1 distance_1 from here but with heavy traffic on our way. <user> give me the address. <system> the address to poi_1 is address_1. <user> thank you car, please pick the quickest way to get there!']

['<user> give me directions to distance_1 poi_type_1.',
 '<user> give me directions to distance_1 poi_type_1. <system> there is poi_1 distance_2 from here but with heavy traffic on our way. <user> give me the address.',
 '<user> give me directions to distance_1 poi_type_1. <system> there is poi_1 distance_2 from here but with heavy traffic on our way. <user> give me the address. <system> the address to poi_1 is address_1. <user> thank you car, please pick the quickest way to get there!']

['give me directions to distance_1 poi_type_1.',
 'give me the address of the poi_type_1 poi_1 distance_1.',
 'thank you car, please pick the quickest way to get to the poi_type_1 poi_1 distance_1 at address_1!']

In [8]:
from sklearn.model_selection import train_test_split 
for dataset in ['TREC++_Entity_index']:
    with open('Data/Model/'+dataset +'_input.txt', 'r') as file:
        all_input = np.array(file.readlines())
    with open('Data/Model/Output/'+dataset +'_output.txt', 'r') as file:
        all_output = np.array(file.readlines())
        
    X_train, X_test, y_train, y_test = train_test_split(all_input, all_output, test_size=.30)
    print(len(X_train), len(X_test))
    for extension, X, y in tqdm([('TRAIN', X_train, y_train), ('TEST', X_test, y_test)]):
        with open('Data/Model/'+dataset + '_'+ extension+'_input.txt', 'w') as f:
            print(*X, file=f)
        with open('Data/Model/Output/'+dataset + '_'+ extension+'_output.txt', 'w') as f:
            print(*y, file=f)

101 44


100%|█████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 154.28it/s]


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(u"Where is the car you said was mine, or is it yours")

for ent in doc.ents:
    print('Entities', ent.text, ent.start_char, ent.end_char, ent.label_)
    
for chunk in doc.noun_chunks:
    print('Chunks', chunk.text, chunk.root.dep_)
    
for token in doc:
    print('Pos', token, token.pos_)

Chunks the car nsubj
Chunks you nsubj
Chunks it nsubj
Pos Where ADV
Pos is VERB
Pos the DET
Pos car NOUN
Pos you PRON
Pos said VERB
Pos was VERB
Pos mine PRON
Pos , PUNCT
Pos or CCONJ
Pos is VERB
Pos it PRON
Pos yours ADJ


In [14]:
import os
def getName(file):
    """
    Extracts name from file
    :param turn: file
    :return: name
    """
    name = ""
    name += 'Transformer ' if "TRAN" in file else "RNN " if "RNN" in file else ""
    name += "ALL" if "ALL" in file else "Vanilla" if "VAN" in file else "Role Based Entity" if name else ""
    return name if name else file[:-4]

allData = [(getName(file), "Results/logs/"+file) 
           for file in os.listdir("Results/logs")]
allData

[('RNN ALL', 'Results/logs/RNN_ALL_log.txt'),
 ('RNN Role Based Entity', 'Results/logs/RNN_GLOVE_RBEI_log.txt'),
 ('RNN Role Based Entity', 'Results/logs/RNN_RBEI_log.txt'),
 ('RNN Vanilla', 'Results/logs/RNN_VAN_log.txt'),
 ('Transformer ALL', 'Results/logs/TRAN_ALL_log.txt'),
 ('Transformer Role Based Entity', 'Results/logs/TRAN_RBEI_log.txt'),
 ('Transformer Vanilla', 'Results/logs/TRAN_VAN_log.txt')]

In [56]:
index = randint(0, 241)
slots = pd.read_json('Data/Table/DEV CQR Table.json').Slots[index]
display(index, slots)
single_slot = reduce(lambda y, x: y+x, 
                     [[entity for entity in entities] 
                      for slot, entities in sorted(slots.items())])
single_slot

110

{'location': ['new york'], 'date': ['friday'], 'weather_attribute': ['humid']}

['friday', 'new york', 'humid']

In [48]:
from functools import reduce
a = reduce(lambda y, x: y, sorted(slots.items())) 
a

('distance', ['within 5 miles', '2 miles away'])

In [7]:
for dataset in ['TRAIN', 'TEST', 'DEV', 'TREC'][:3]:
    with open('Data/Model/'+dataset + '_ALL_D2T_input.txt', 'r') as file:
        print(dataset, len(file.readlines()))

TRAIN 6019
TEST 784
DEV 744


In [2]:
data = dict()
all_types = set()
for dataset in ['train', 'test', 'dev']:
    types = Counter()
    slots = pd.read_json('Data/Table/{} CQR Table.json'.format(dataset.upper())).Slots
    for slot in slots:
        types += Counter(slot.keys())
    data[dataset.upper()] = types
    all_types.update(types.keys())
table = defaultdict(list)
for dataset in data:
    table['Name'].append(dataset)
    total = 1#sum(data[dataset].values())*1.0
    for types in all_types:
        table[types].append(data[dataset][types]/total)
pd.DataFrame.from_dict(table).to_excel('Results/Analysis/Slot Types count.xlsx', index=False)
pd.DataFrame.from_dict(table)

,Name,date,party,room,weather_attribute,poi_type,location,address,day,traffic_info,time,agendas,event,agenda,poi,distance
0,TRAIN,1201.0,329.0,29.0,704.0,758.0,787.0,577.0,1.0,503.0,428.0,1.0,544.0,65.0,768.0,739.0
1,TEST,137.0,30.0,0.0,79.0,94.0,100.0,57.0,0.0,49.0,42.0,0.0,76.0,4.0,96.0,87.0
2,DEV,139.0,33.0,3.0,87.0,94.0,97.0,43.0,0.0,48.0,50.0,0.0,72.0,7.0,93.0,81.0


In [5]:
Turns = Counter()
for dataset in ['train', 'test', 'dev']:
    dialogs = pd.read_json('Data/Table/{} CQR Table.json'.format(dataset.upper())).Original_turns
    for dialog in dialogs:
        Turns[dataset] += len(dialog)
    Turns[dataset] /= len(dialogs)
Turns

Counter({'train': 5.633974659784139,
         'test': 5.670289855072464,
         'dev': 5.479704797047971})